<a href="https://colab.research.google.com/github/suryasphoorthi536-rgb/Automated-Underwriting-Platform/blob/main/Smart_Claims_Processing_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install pytesseract pdf2image opencv-python transformers langchain fastapi uvicorn
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [45]:
from google.colab import files
# from pdf2image import convert_from_path # Removed since we are processing images
import cv2
import pytesseract
import os
import numpy as np # Import numpy

# Upload files
uploaded = files.upload()

# Check if a file was uploaded and get the filename
if not uploaded:
    print("No file uploaded.")
else:
    filename = next(iter(uploaded))
    print(f"Processing file: {filename}")

    # Read the image file
    # Use cv2.imdecode to read image from bytes
    image = cv2.imdecode(np.frombuffer(uploaded[filename], np.uint8), cv2.IMREAD_COLOR)


    if image is None:
        print(f"Error: Could not read image file {filename}. Please check the file format.")
    else:
        # Preprocess the image
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)[1]

        # OCR
        text = pytesseract.image_to_string(image)
        print("\n--- Extracted Text ---")
        print(text)

# If you need to process PDF files in the future, you'll need to install poppler.
# You can do this by running the following command in a code cell:
# !apt-get install poppler-utils -y

Saving resume.pdf to resume (1).pdf
Processing file: resume (1).pdf
Error: Could not read image file resume (1).pdf. Please check the file format.


In [54]:
from transformers import pipeline

nlp = pipeline("ner", model="dslim/bert-base-NER")
entities = nlp(text)

for entity in entities:
    print(entity)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


{'entity': 'B-ORG', 'score': np.float32(0.9980804), 'index': 1, 'word': 'Tamil', 'start': 0, 'end': 5}
{'entity': 'I-ORG', 'score': np.float32(0.99863535), 'index': 2, 'word': 'Q', 'start': 7, 'end': 8}
{'entity': 'I-ORG', 'score': np.float32(0.9967158), 'index': 3, 'word': '##ual', 'start': 8, 'end': 11}
{'entity': 'I-ORG', 'score': np.float32(0.99718994), 'index': 4, 'word': '##ities', 'start': 11, 'end': 16}
{'entity': 'I-ORG', 'score': np.float32(0.6844323), 'index': 25, 'word': 'Ana', 'start': 130, 'end': 133}
{'entity': 'I-ORG', 'score': np.float32(0.5808866), 'index': 26, 'word': '##ly', 'start': 133, 'end': 135}
{'entity': 'B-ORG', 'score': np.float32(0.5060131), 'index': 30, 'word': 'IBM', 'start': 151, 'end': 154}
{'entity': 'I-MISC', 'score': np.float32(0.36933863), 'index': 31, 'word': '‘', 'start': 155, 'end': 156}
{'entity': 'I-MISC', 'score': np.float32(0.75211954), 'index': 32, 'word': 'Skills', 'start': 156, 'end': 162}
{'entity': 'I-MISC', 'score': np.float32(0.908336

In [61]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier object
# Using random_state for reproducibility
model = RandomForestClassifier(random_state=42)

# Train the model using the .fit() method on your training data
model.fit(X_train, y_train)

print("Random Forest Classifier model trained successfully.")

Random Forest Classifier model trained successfully.


In [65]:
def route_claim(claim_data):
    if claim_data["type"] == "Auto":
        return "Routing to Auto Dept"
    elif claim_data["type"] == "Health":
        return "Routing to Health Dept"
    return "General Claims Queue"


**Reasoning**:
Create a sample list of dictionaries to simulate claim data with claim types.



In [66]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class ClaimRequest(BaseModel):
    text: str

@app.post("/process_claim/")
def process_claim(request: ClaimRequest):
    # process and route logic here
    return {"status": "processed", "routing": route_claim({"type": "Auto"})}


In [68]:
import json

with open("claim_output.json", "w") as f:
    json.dump(output_data, f)

In [32]:
!pip install nest_asyncio pyngrok uvicorn

import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading

# Apply nest_asyncio to allow multiple event loops
nest_asyncio.apply()

# Terminate any active ngrok tunnels
print("Terminating active ngrok tunnels...")
ngrok.kill()

# Set up ngrok tunnel
NGROK_AUTH_TOKEN = "YOUR_AUTH_TOKEN" # Replace with your ngrok auth token
# You can get an auth token from https://dashboard.ngrok.com/get-started/your-authtoken
# Add it to Colab secrets or replace directly for demonstration

if NGROK_AUTH_TOKEN == "YOUR_AUTH_TOKEN":
    print("Please replace 'YOUR_AUTH_TOKEN' with your actual ngrok authentication token.")
else:
    print("Setting up ngrok tunnel...")
    try:
        # Set ngrok auth token
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        # Open a tunnel to the default uvicorn port (8000)
        public_url = ngrok.connect(8000).public_url
        print(f"FastAPI app available at: {public_url}")

        # Run uvicorn in a separate thread
        def run_uvicorn():
            uvicorn.run(app, host="0.0.0.0", port=8000)

        uvicorn_thread = threading.Thread(target=run_uvicorn)
        uvicorn_thread.daemon = True
        uvicorn_thread.start()

    except Exception as e:
        print(f"An error occurred setting up ngrok or running uvicorn: {e}")

Terminating active ngrok tunnels...
Please replace 'YOUR_AUTH_TOKEN' with your actual ngrok authentication token.


In [73]:
import os

# Create directories
os.makedirs("/content/claims_ai_project/data", exist_ok=True)
os.makedirs("/content/claims_ai_project/models", exist_ok=True)
os.makedirs("/content/claims_ai_project/notebooks", exist_ok=True)
os.makedirs("/content/claims_ai_project/outputs", exist_ok=True)


In [34]:
# Check if output_data exists and is not empty
if 'output_data' in locals() and output_data:
    print("output_data exists and contains data.")
    # Display the contents of output_data
    display(output_data)
else:
    print("output_data is not available or empty. Please run the previous subtask 'Structure output data'.")


output_data exists and contains data.


[{'claim_info': 'Car accident on main street',
  'claim_type': 'Auto',
  'routing': 'Routing to Auto Dept'},
 {'claim_info': 'Doctor visit for checkup',
  'claim_type': 'Health',
  'routing': 'Routing to Health Dept'},
 {'claim_info': 'Water damage in basement',
  'claim_type': 'Property',
  'routing': 'General Claims Queue'},
 {'claim_info': 'Stolen car reported',
  'claim_type': 'Auto',
  'routing': 'Routing to Auto Dept'},
 {'claim_info': 'Emergency room visit',
  'claim_type': 'Health',
  'routing': 'Routing to Health Dept'},
 {'claim_info': 'Fire damage in garage',
  'claim_type': 'Property',
  'routing': 'General Claims Queue'}]